# Imports

In [ ]:
import sys
!{sys.executable} -m pip install transformers datasets tokenizers evaluate
import datasets

In [8]:
data = datasets.load_dataset('webis/Touche23-ValueEval')
data["training"].to_pandas().head()


Using custom data configuration webis--Touche23-ValueEval-6979f60cf05ee731
Found cached dataset csv (/Users/lisakoopmans/.cache/huggingface/datasets/webis___csv/webis--Touche23-ValueEval-6979f60cf05ee731/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

,Argument ID,Conclusion,Stance,Premise,Labels
0,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...,"[0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]"
1,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...,"[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]"
2,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...,"[0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0]"
3,A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...,"[0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0]"
4,A01008,We should ban factory farming,against,factory farming allows for the production of c...,"[0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0]"
